### Named Entity Extraction

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import re
import PyPDF2

In [5]:
nlp = en_core_web_sm.load()

doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile'
          ' phone market and ordered the company to alter its practices')


pprint([(X.text, X.label_) for X in doc.ents])
# pprint([(X, X.ent_iob_, X.ent_type_) for X in doc])

[('European', 'NORP'),
 ('Google', 'ORG'),
 ('$5.1 billion', 'MONEY'),
 ('Wednesday', 'DATE')]


In [7]:
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [10]:
ny_bb = url_to_string(
    'https://www.nytimes.com/2018/08/13/us/politics/peter-strzok-fired-fbi.html?hp&action=click&pgtype=Homepage'
    '&clickSource=story-heading&module=first-column-region&region=top-news&WT.nav=top-news')
article = nlp(ny_bb)
print(article)

     F.B.I. Agent Peter Strzok, Who Criticized Trump in Texts, Is Fired - The New York Times                                                                                                             SectionsSEARCHSkip to contentSkip to site indexPoliticsLog inToday’s PaperPolitics|F.B.I. Agent Peter Strzok, Who Criticized Trump in Texts, Is Firedhttps://nyti.ms/2OtNre3AdvertisementContinue reading the main storySupported byContinue reading the main storyF.B.I. Agent Peter Strzok, Who Criticized Trump in Texts, Is FiredPeter Strzok, a top F.B.I. counterintelligence agent who was taken off the special counsel investigation after his disparaging texts about President Trump were uncovered, was fired.Credit...T.J. Kirkpatrick for The New York TimesBy Adam Goldman and Michael S. SchmidtAug. 13, 2018WASHINGTON — Peter Strzok, the F.B.I. senior counterintelligence agent who disparaged President Trump in inflammatory text messages and helped oversee the Hillary Clinton email and Russia invest

In [17]:
# Create a file to extract the text from a pdf file
# Figure out a better way to select a file
def parsePdf(file):
    with open(file, 'rb') as pdfFileObj:
        # Create the file reader object
        # This operation can take some time, as the PDF stream’s cross-reference tables are read into memory
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        print(pdfReader.numPages)
        allText = ''
        for i in range(int(pdfReader.numPages)):
            pageObj = pdfReader.getPage(i)
            allText += pageObj.extractText()
        return allText

In [44]:
# prompt the user to get the file from directory
import tkinter as tk
from tkinter import filedialog
def getFile():
    root = tk.Tk()
    root.withdraw()

    file_path = filedialog.askopenfilename()
    return file_path

In [15]:
file_path = getFile()
print(file_path)

NameError: name 'getFile' is not defined

In [18]:
# if file_path:
#     file = file_path
# else:
file = '/Users/peterargo/PycharmProjects/pythonProject2/pdfs/Dr. Faucci.pdf'
allText = parsePdf(file)

1


In [19]:
print(allText)

Washington (CNN)
Dr. Anthony Fauci said Friday that the lack of truthfulness from the
 Trump administration regarding the Covid
-19 pandemic "very likely" cost American lives.
 "Particularly when you're in the situation of almost being in a crisis with the number of cases and hospitalizations and deaths that we have -- when you start talking about things that make no sense medically and no sense scientifically, that clearly is not helpful," Fauci, the chief medical adviser to President Joe Biden, said on CNN's "New Day" Friday. Asked by CNN's John Berman if the lack of candor over the last year and lack of facts, in some cases, cost lives, Fauci said, "You know, it very likely did." He warned that it's "not helpful" when "you're starting to go down paths that are not based on any science at all," adding that he doesn't wish to rehash the ways in which the Trump administration steered away from science. BIDEN'S FIRST 100 DAYS
 ¥!Biden unveils Covid
-19 plan based on 'science not politic

In [20]:
# Put all the sentences in a list
sentences = [x for x in article.sents]

# Get a random sentence
randomSentence = sentences[22]

# Display the raw markup of the entire article
displacy.render(nlp(str(article.doc)), jupyter=True, style='ent')

In [10]:
displacy.render(nlp(str(sentences[22])), style='dep', jupyter = True, options = {'distance': 120})

In [21]:
# Display the raw markup of the entire PDf
displacy.render(nlp(str(allText)), jupyter=True, style='ent')

In [22]:
displacy.render(nlp(str(allText)), style='dep', jupyter = True, options = {'distance': 120})

### Image Extraction

In [54]:
from PIL import Image

In [71]:
def parsePdf_Image(file):
    with open(file, 'rb') as pdfFileObj:
        # Create the file reader object
        # This operation can take some time, as the PDF stream’s cross-reference tables are read into memory
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        print(pdfReader.numPages) # Vector check to see if I am reading the expected PDF
        
        # Get the first page for now
        page0 = pdfReader.getPage(0)
        xObject = page0['/Resources']['/ProcSet'].getObject()
        return xObject

In [72]:
if file_path:
    file = file_path
else:
    file = '/Users/peterargo/PycharmProjects/TestPdf.pdf'
xObject = parsePdf_Image(file)

2


In [81]:
# Get all files in a directory
import glob
file_path = getFile()
print(file_path)
print(glob.glob("/home/adam/*.txt"))


[]


In [94]:
# This gets all the filles in a path that is definied in the code
# Make this recursive 
import tkinter as Tk
from tkinter import filedialog
from os import listdir
from os.path import isfile, join


def get_pdfs(file_path):
    myPath = file_path + '/'
    onlyfiles = [myPath + f for f in listdir(myPath) if (isfile(join(myPath, f)) and f.find('.pdf') != -1) else get_pdfs(myPath + f)]
    print(onlyfiles)

root = tk.Tk()
root.withdraw()
folder_selected = filedialog.askdirectory()
print(folder_selected)

get_pdfs(folder_selected)

SyntaxError: invalid syntax (<ipython-input-94-c897fd8e63f2>, line 11)

In [2]:
# This gets all the filles in a path that is definied in the code
# This is recursive 
# Can this be more efficient?
import tkinter as tk
from tkinter import filedialog
from os import listdir
from os.path import isfile, join


def get_pdfs(file_path):
    myPath = file_path + '/'
    print(myPath)
    onlyfiles = list()
    for f in listdir(myPath):
        if (isfile(join(myPath, f)) and f.find('.pdf') != -1):
            onlyfiles.append(join(myPath, f))
        elif not isfile(join(myPath, f)):
            recursive_dir = get_pdfs(join(myPath, f))
            for f1 in recursive_dir:
                onlyfiles.append(f1) 
    return onlyfiles
        
root = tk.Tk()
root.withdraw()
folder_selected = filedialog.askdirectory()

pdfs = get_pdfs(folder_selected)
print(pdfs)

/
/home/
/usr/
/usr/bin/
/usr/standalone/
/usr/standalone/firmware/
/usr/standalone/firmware/t302/
/usr/standalone/firmware/SLAM/
/usr/standalone/firmware/iBridge1_1Customer.bundle/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/_CodeSignature/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/Resources/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/Resources/Firmware/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/Resources/Firmware/usr/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/Resources/Firmware/usr/local/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/Resources/Firmware/usr/local/standalone/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/Resources/Firmware/dfu/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/Resources/Firmware/all_flash/
/usr/standalone/firmware/iBridge1_1Customer.bundle/Contents/Resources/Firmware/al

PermissionError: [Errno 13] Permission denied: '/usr/sbin/weakpass_edit/'

In [74]:
for obj in xObject:
    if obj == '/Image':
        size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
        data = xObject[obj].getData()
        if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
            mode = "RGB"
        else:
            mode = "P"

        if xObject[obj]['/Filter'] == '/FlateDecode':
            img = Image.frombytes(mode, size, data)
            img.save(obj[1:] + ".png")
        elif xObject[obj]['/Filter'] == '/DCTDecode':
            img = open(obj[1:] + ".jpg", "wb")
            img.write(data)
            img.close()
        elif xObject[obj]['/Filter'] == '/JPXDecode':
            img = open(obj[1:] + ".jp2", "wb")
            img.write(data)
            img.close()

In [ ]:
# Try with PyMuPDF Pillow

In [1]:
import fitz # PyMuPDF
import io
from PIL import Image

In [78]:
file = '/Users/peterargo/PycharmProjects/TestPdf.pdf'
# open the file
pdf_file = fitz.open(file)

In [10]:
image_index = 1
for file in pdfs:
    pdf_file = fitz.open(file)
    for page_index in range(len(pdf_file)):
        # get the page itself
        page = pdf_file[page_index]
        image_list = page.getImageList()
        # printing number of images found in this page
        if image_list:
            print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
        else:
            print("[!] No images found on page", page_index)
        for img in page.getImageList():
            # get the XREF of the image
            xref = img[0]
            # extract the image bytes
            base_image = pdf_file.extractImage(xref)
            image_bytes = base_image["image"]
            # get the image extension
            image_ext = base_image["ext"]
            # load it to PIL
            image = Image.open(io.BytesIO(image_bytes))
            # save it to local disk
            image.save(open(f"image{page_index+1}_{image_index}.{image_ext}", "wb"))
            # increment the image index
            image_index += 1

[+] Found a total of 1 images in page 0
[+] Found a total of 1 images in page 1
[!] No images found on page 0
[!] No images found on page 1
[+] Found a total of 1 images in page 0
[+] Found a total of 1 images in page 1


In [2]:
mas = ['a']
if mas:
    print('yes')

yes
